# Simulação CTA com termistor pulsado

Um CTA (*Constant Temperatura Anemometer*) é um anemômetro onde um elemento aquecido cuja resistência varia com a temperatura é mantido a uma temperatura (e portanto resistência constante). Este notebook tem por objetivo simular um um anemômetro pulsado onde o controle da resistência é feita por um sinal PWM (*Pulse Width Modulation*) e a entrada analógica de um microcontrolador. O controle é feito por software no microcontrolador.

O circuito funciona usando o sinal PWM do microcontrolador para ligar e desligar um circuito com um termistor e um resistor. A tensao neste resistor passa por um filtro passa baixa onde um sinal DC e usado para se estimar a resistencia (e portanto a temperatura) do termistor. Esta tensao por sua vez e lida em uma entrada analogica do microncontrolador. Software no microcontrolador modifica o *duty cycle* do sinal PWM para controlar a resistência do PWM.

In [ ]:
using Plots
using DifferentialEquations
gr()

In [ ]:
include("../src/ThermistorHW.jl")
using ThermistorHW

In [ ]:
include("cta-pulsed.jl")

In [ ]:
Rt = Thermistor(5e3, 3200)
cta = CTA.PulsedCTA(Rt)
Dmm = 2.0
htrans = CTA.HeatTrans(Dmm)
mcp = CTA.mcpfun(Fe₂O₃, htrans.D)
ufun(t) = 10.0
duty = 0.2

In [ ]:
y0 = [0.0, htrans.Ta]
prob = ODEProblem(CTA.pulsedcta, y0, (0.0, 30.0), 
    (cta, duty, ufun, htrans, mcp))

In [ ]:
sol = solve(prob, Tsit5(), reltol=1e-8);

In [ ]:
Eai = [u[1] for u in sol.u];
T = [u[2] for u in sol.u];

In [ ]:
plot(sol.t, T)

In [ ]:
plot(sol.t, Eai)